# Proyecto Natural Language Processing (NLP):

In [63]:
# Librería para la declaración y uso de Data Frames:
import pandas as pd

# Librería para la utilización de Expresiones Regulares:
import re

# Librería para la utilización de herramientas para NLP:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/vscode/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Paso 1 - Lectura de Datos: 

En primer lugar, es necesario **leer y guardar la información** en una variable para poder empezar a trabajar con ella.

Para ello, se ha guaradado el archivo con todos los datos en la ruta: */workspaces/NLP-clara-ab/data/raw/url_spam.csv* y se ha cargado en un Data Frame:

In [64]:
# Lectura del CSV con los datos, dada la ruta donde se guarda el archivo (se ha evitado cargar la primera columna con los índices de las filas):
df = pd.read_csv ('/workspaces/NLP-clara-ab/data/raw/url_spam.csv', sep = ",");

# Configuración de pandas para mostrar todas las columnas del DataFrame sin truncarlas al visualizarlo:
pd.set_option('display.max_columns', None);

# Se muestran las 5 primeras filas del Data Frame:
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


Una vez se ha cargado correctamente la información en el Data Frame df es interesante evaluar la **cantidad de información que se tiene**. Para ello, se recurre al atributo `shape` del Data Frame:

In [65]:
# Se utiliza el atributo shape del Data Frame para conocer cuánta información está cargada:
print(f" El archivo cargado contiene {df.shape[0]} URLs"); 

 El archivo cargado contiene 2999 URLs


Además, tal y como se ha visto en el **breve resumen** mostrado a partir de `.head()` la variable a predecir, `is_spam` tiene dos valores posibles: `True` or `False`. 

En este caso, nos interesa predecir si una URL es SPAM, es decir, **detectar el `TRUE`**. Por lo que se va a realizar un cambio de forma que, en todas las filas donde la columna `is_spam` sea `True`, se convertirán a `1` y donde haya un `False` se convertirán a `0`. 

Para no tener problemas, es importante comprobar, primero el **tipo de variable** que se encuentra en la columna `is_spam`:

In [66]:
# Se imprimen los tipos de variables de las dos columnas del DataFrame:
print(df.dtypes)

url        object
is_spam      bool
dtype: object


Ahora que se sabe que es un **boolean**, se puede realizar la conversión con la función `map()` de forma segura:

In [67]:
# Se codifica la columna objetivo:
df['is_spam'] = df['is_spam'].astype(bool).map({True: 1, False: 0})

# Se comprueba que se ha codificado correctamente: 
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


## Paso 2 - Análisis Exploratorio de Datos (Previo):

Antes de pasar el preprocesado de texto, donde se prepara al conjunto de datos para el análisis, es necesario realizar un **análisis exploratorio de los datos previo** de forma que se puedan **eliminar redundancias o datos faltantes** y no trabajar sobre ellos de forma innecesaria en el paso siguiente. 


### Paso 2.1 - Análisis de Duplicados:

In [68]:
# Se utiliza el método .duplicated() para identificar las URL repetidas dentro del DataFrame:
print(f" Hay un total de {df.duplicated().sum()} URLs duplicadas.");

 Hay un total de 630 URLs duplicadas.


Se han encontrado un total de 630 URLs duplicadas que se deben eliminar:

In [69]:
# Se eliminan los duplicados:
df_clean = df.drop_duplicates();

# Se comprueba que ya no quedan duplicados:
print(f" Hay un total de {df_clean.duplicated().sum()} URLs duplicadas.");

 Hay un total de 0 URLs duplicadas.


### Paso 2.2 - Análisis de Nulos:

In [70]:
# Se comprueba el porcentaje de filas que presentan al menos un valor nulo:
print(f" El {round(df_clean.isnull().any(axis=1).mean()*100, 2)} % de las filas presentan, al menos, un valor nulo");

 El 0.0 % de las filas presentan, al menos, un valor nulo


No tenemos ninguna fila con valores nulos.

## Paso 3 - Preprocesamiento de Texto:

Una vez ya se tiene guardado el conjunto de datos con una breve **limpieza previa**, se puede pasar a realizar un **preprocesado del texto a estudiar**, en este caso  las URLs.

El **preprocesamiento de texto** es un paso **crucial** en el análisis de información textual, dado que ayuda a **transformar la información en bruto en un formato más estructurado y útil** para modelos de Machine Learning. De esta forma, se ayuda a eliminar el *ruido* desechando los patrones, *a priori*, irrelevantes. 

Para ello, se van a utilizar **dos librerías** imprescindibles en el procesado de texto: 

**- `re`:** Librería de Python utilizada para trabajar con **expresiones regulares** (secuencia de caracteres que definen un patrón de búsqueda). Es útil para **segementar y limpiar textos** mediante diferentes patrones. En este caso, va a permitir dividir las URLs por caracteries especiales y eliminar todos los elementos no deseados. 

**- `nltk` Natural Language Toolkit:** Librería de Python con **herramientas** útiles para el procesado de lenguaje natural como *stopwords*, lematización y tokenización, lo que facilita la **normalización** del texto antes de aplicar cualquier modelo de Machine Learning. 

En este caso, se van a realizar 7 pasos: 

1. Conversión del texto a minúscula

2. Segmentación de las URLs

3. Eliminado de palabras sin valor y espacios

4. Tokenización

5. Eliminado de *StopWords*

6. Lematización



Para poder realizar todo este preprocesado, se va a utilizar una **función** que reciba como input el texto a preprocesar y se le devuelva totalmente procesado:

In [136]:
def preprocess_text (url):

    # 1. Conversión a minúsculas:
    url = url.lower();
      
    # 2. Eliminado de palabras irrelevantes:
    url = re.sub(r'https?://|www\.', '', url); # Protocolo web
    url = re.sub(r'\s+[a-zA-Z]\s+', " ", url) # Espacios en blanco
    url = re.sub(r'\s*\d+\s*', ' ', url)  # Elimina números aislados
    url = re.sub(r'\b[a-zA-Z]*\d+[a-zA-Z]*\b', ' ', url)  # Borra tokens con mezcla de letras y números
    
    # Solo texto
    url = re.sub(r'[^\w\s]|[\d]', ' ', url)
    # Eliminación de los espacios adicionales
    url = re.sub(r'\s+', ' ', url).strip()

    # 3. Segmentación de las URLs:
    tokens = re.split(r'[\/\.\?\=\&\-_]', url);


    
    # Tokenización del texto
    #tokens = word_tokenize(url)

    # 6. Eliminación de stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lematización de palabras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

In [140]:
# Ejemplo de aplicación de la función de preprocesado para un tweet aleatorio
url_random = df_clean.sample(1).url.values[0]
print(f'URL antes de ser preprocesada:\n {url_random}')
url_random_prepro = preprocess_text(url_random)
print('-'*50)
print(f'URL tras ser preprocesado:\n {url_random_prepro}')

URL antes de ser preprocesada:
 https://cloudwars.co/covid-19/zoom-quarter-10-eye-popping-stats-from-techs-new-superstar/
--------------------------------------------------
URL tras ser preprocesado:
 cloudwars co covid zoom quarter eye popping stats from techs new superstar


### Paso 3.1 - Conversión a Minúscula:

La** conversión de la URLs a minúsculas** asegura que las variaciones en el caso de las letras no afecten al análisis´. El hecho de que una palabra lleve o no minúscula, **no aporta información relevante** a la clasificación por lo que es mejor trabajar con una **uniformidad en los datos**. 

Para conseguirlo, se va a utilizar la función `lower()` sobre la columna de URLs.


In [73]:
# Se pasa todo a minúsculas:
df_

NameError: name 'df_' is not defined